In [1]:
import re
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
chikan = re.compile('[ (\n)]')

In [3]:
def startUp():
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [4]:
TopUrl = "https://highlow.com/"

In [16]:
driver = startUp()
driver.get(TopUrl)

In [20]:
quickDemo = "body > div.page-content > div.status-bar.sticky-strip > div > div > span > span > a"
click = driver.find_element(By.CSS_SELECTOR, quickDemo)
click.click()

In [22]:
answer = "#root > div > div.SecondaryBanner_container__1dK5N.GuidedTourInvitationBanner_readyToTradeBanner__Zx9Vd.SecondaryBanner_enter__1PYvo > div > div.SecondaryBanner_closeButton__1iz3l > svg"
click = driver.find_element(By.CSS_SELECTOR, answer)
click.click()

In [23]:
highButton = "#TradePanel_oneClickHighButton__3OAFf > div"
lowButton = "#TradePanel_oneClickLowButton__3Oq9p > div"
oneclick = "#scroll_panel_1_content > div.App_tradeArea__1rrhJ > div > div.App_tradeForm__3vFhv > div > div.TradePanel_footer__37dl2 > div.TradePanel_label__2ivNV > div > svg"
driver.find_element(By.CSS_SELECTOR, oneclick).click()

In [11]:
def GetPrice(driver):
    priceInfo = "#scroll_panel_1_content > div.App_tradeArea__1rrhJ > div > div.App_chart__1TMEu > div.ChartInfo_chartInfo__2JqPK.ChartInfo_container__1N5nc.App_chartInfo__FBBmM > div.ChartInfo_optionAssetContainer__3bPBB > div > div:nth-child(2)"
    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.select_one(priceInfo)
    return float(chikan.sub("", tables.text))

In [12]:
def Retry(driver, Button):
    while True:
        driver.find_element(By.CSS_SELECTOR, Button).click()
        time.sleep(0.125)
        html = driver.page_source.encode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        tables = soup.select_one("#root > div > div.Toasts_container__3FULM.App_toasts__2BOdb").text
        tables = chikan.sub("", tables)
        if len(tables) == 0:
            break

In [25]:
index = 0
while True:
    try:
        if random.randint(0, 1):
            Retry(driver, highButton)
        else:
            Retry(driver, lowButton)
        time.sleep(30)
    except KeyboardInterrupt:
        break
    except:
        time.sleep(0.125)
        continue
    if index == 10:
        break
    index += 1

In [14]:
cookies = driver.get_cookies()

In [21]:
for cookie in cookies: driver.add_cookie(cookie)

In [26]:
driver.quit()